In [1]:
import numpy as np
from random import shuffle
# import scipy.ndimage
import scipy.io as io
import pandas as pd
# import argparse

In [3]:
nval = 200
nclasses = 9

test_datamat = io.loadmat('indian_pines_Test_patch_3.mat')
test_data = test_datamat['test_patch']
test_labels = test_datamat['test_labels'].transpose()
# channels = test_data.shape[2] #test_data.shape[1]

train_datamat = io.loadmat('indian_pines_Train_patch_3.mat')
train_data = train_datamat['train_patch']
train_labels = train_datamat['train_labels'].transpose()

val_data = io.loadmat("indian_pines_Val_patch_3.mat")['val_patch']
val_labels = io.loadmat("indian_pines_Val_patch_3.mat")['val_labels'].transpose()

In [4]:
trainData = train_data.reshape(1600, 220*3*3)

In [11]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=9)
neigh.fit(trainData, train_labels) 

/home/ajay/anaconda3/envs/baseml/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                     weights='uniform')

In [12]:
preVal = neigh.predict(val_data.reshape(200, 220*3*3))
preTest = neigh.predict(test_data.reshape(7434, 220*3*3))

In [13]:
from sklearn.metrics import accuracy_score

print("Val score:", accuracy_score(preVal, val_labels))
print("Test score:", accuracy_score(preTest, test_labels))

Val score: 0.725
Test score: 0.68348130212537


In [14]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
def summary(t1, t2):
    matrix = confusion_matrix(t1, t2)
    matrixDict = {}
    z = matrix.diagonal()/matrix.sum(axis=1)
    for i in range(9):
        matrixDict[str(i)] = z[i]
    matrixDict["macro avg"] = accuracy_score(t1, t2)
    matrixDict["weighted avg"] = accuracy_score(t1, t2)
    accuracydf = pd.DataFrame(list(matrixDict.items()), columns=["class", "accuracy"]) 

    report = classification_report(t1, t2, output_dict=True)
    metricsdf = pd.DataFrame(report)
    metricsdf = metricsdf.drop(columns = ["accuracy"])
    metricsdf = metricsdf.transpose()
    metricsdf = metricsdf.drop(columns = ["support"])
    metricsdf['class'] = metricsdf.index
    metricsdf = metricsdf[["class", "precision", "recall", "f1-score"]]
    metricsdf = pd.merge(metricsdf, accuracydf, on=['class'])
    return accuracydf, metricsdf

In [47]:
accdf1, metdf1 = summary(val_labels, preVal)

In [48]:
metdf1.round(2)

,class,precision,recall,f1-score,accuracy
0,0,0.46,0.55,0.50,0.55
1,1,0.60,0.55,0.57,0.55
2,2,0.77,0.91,0.83,0.91
3,3,0.88,0.96,0.92,0.96
4,4,1.00,1.00,1.00,1.00
5,5,0.71,0.77,0.74,0.77
6,6,0.42,0.50,0.46,0.50
7,7,0.86,0.55,0.67,0.55
8,8,1.00,0.73,0.84,0.73
9,macro avg,0.74,0.72,0.73,0.72


## RF

In [19]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(trainData, train_labels.ravel())

/home/ajay/anaconda3/envs/baseml/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [20]:
preVal2 = rf.predict(val_data.reshape(200, 220*3*3))
preTest2 = rf.predict(test_data.reshape(7434, 220*3*3))

In [33]:
print("Val score:", accuracy_score(preVal2, val_labels))
print("Test score:", accuracy_score(preTest2, test_labels))

Val score: 0.78
Test score: 0.7313693839117568


In [22]:
accdf2, metdf2 = summary(val_labels, preVal2)
metdf2.round(2)

,class,precision,recall,f1-score,accuracy
0,0,0.60,0.68,0.64,0.68
1,1,0.67,0.73,0.70,0.73
2,2,0.95,0.95,0.95,0.95
3,3,0.92,1.00,0.96,1.00
4,4,1.00,1.00,1.00,1.00
5,5,0.78,0.64,0.70,0.64
6,6,0.46,0.50,0.48,0.50
7,7,0.67,0.55,0.60,0.55
8,8,1.00,0.95,0.98,0.95
9,macro avg,0.78,0.78,0.78,0.78


## SVM

In [24]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(trainData, train_labels.ravel())

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [25]:
preVal3 = clf.predict(val_data.reshape(200, 220*3*3))
preTest3 = clf.predict(test_data.reshape(7434, 220*3*3))

In [26]:
accdf3, metdf3 = summary(preVal3, val_labels)

/home/ajay/anaconda3/envs/baseml/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  
/home/ajay/anaconda3/envs/baseml/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [27]:
metdf3

,class,precision,recall,f1-score,accuracy
0,0,0.863636,0.211111,0.339286,0.211111
1,1,0.000000,0.000000,0.000000,NaN
2,2,0.000000,0.000000,0.000000,0.000000
3,3,0.391304,0.500000,0.439024,0.500000
4,4,0.000000,0.000000,0.000000,NaN
5,5,0.363636,0.250000,0.296296,0.250000
6,6,0.000000,0.000000,0.000000,0.000000
7,7,0.000000,0.000000,0.000000,0.000000
8,8,1.000000,0.578947,0.733333,0.578947
9,macro avg,0.290953,0.171118,0.200882,0.290000


## XGBoost

In [29]:
from xgboost import XGBClassifier

xg = XGBClassifier()
xg.fit(trainData, train_labels.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [31]:
preVal4 = xg.predict(val_data.reshape(200, 220*3*3))
preTest4 = xg.predict(test_data.reshape(7434, 220*3*3))

In [34]:
print("Val score:", accuracy_score(preVal4, val_labels))
print("Test score:", accuracy_score(preTest4, test_labels))

Val score: 0.835
Test score: 0.789211729889696


In [32]:
accdf4, metdf4 = summary(preVal4, val_labels)

In [37]:
metdf4.round(2)

,class,precision,recall,f1-score,accuracy
0,0,0.64,0.78,0.70,0.78
1,1,0.77,0.81,0.79,0.81
2,2,0.91,0.95,0.93,0.95
3,3,1.00,0.92,0.96,0.92
4,4,1.00,1.00,1.00,1.00
5,5,0.68,0.83,0.75,0.83
6,6,0.68,0.62,0.65,0.62
7,7,0.86,0.66,0.75,0.66
8,8,0.95,1.00,0.98,1.00
9,macro avg,0.83,0.84,0.83,0.84


In [55]:
a = pd.merge(accdf1, accdf2, on=['class'])
a = pd.merge(a, accdf4, on=['class'])
#a = a[]
a = a.rename(columns={"accuracy_x": "kNN", "accuracy_y": "RandomForest", "accuracy": "XGBoost"})
a["class"].iloc[9] = "Overall Accuracy"
a.round(2)
#, colnames=["class", "kNN", "RandomForest"]

/home/ajay/anaconda3/envs/baseml/lib/python3.7/site-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,class,kNN,RandomForest,XGBoost
0,0,0.55,0.68,0.78
1,1,0.55,0.73,0.81
2,2,0.91,0.95,0.95
3,3,0.96,1.00,0.92
4,4,1.00,1.00,1.00
5,5,0.77,0.64,0.83
6,6,0.50,0.50,0.62
7,7,0.55,0.55,0.66
8,8,0.73,0.95,1.00
9,Overall Accuracy,0.72,0.78,0.84


In [52]:
a

,class,accuracy_x,accuracy_y,accuracy
0,0,0.545455,0.681818,0.777778
1,1,0.545455,0.727273,0.809524
2,2,0.909091,0.954545,0.952381
3,3,0.956522,1.000000,0.920000
4,4,1.000000,1.000000,1.000000
5,5,0.772727,0.636364,0.833333
6,6,0.500000,0.500000,0.625000
7,7,0.545455,0.545455,0.655172
8,8,0.727273,0.954545,1.000000
9,macro avg,0.725000,0.780000,0.835000


In [51]:
accdf2

,class,accuracy
0,0,0.681818
1,1,0.727273
2,2,0.954545
3,3,1.000000
4,4,1.000000
5,5,0.636364
6,6,0.500000
7,7,0.545455
8,8,0.954545
9,macro avg,0.780000


In [159]:
train_data.shape

(1600, 220, 3, 3)

In [162]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
model = Sequential()
#add model layers
model.add(Reshape((band_size, patch_size*patch_size)))
model.add(Conv2D(64, kernel_size=3, activation="relu", input_shape=(220,3,3)))
model.add(Conv2D(32, kernel_size=3, activation="relu"))
model.add(Flatten())
model.add(Dense(10, activation="softmax"))

Using TensorFlow backend.


ValueError: Negative dimension size caused by subtracting 3 from 1 for 'conv2d_2/convolution' (op: 'Conv2D') with input shapes: [?,218,1,64], [3,3,64,32].

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)